# Multi-Period Structure Optimization

## Overview
This notebook applies the adjoint-based inverse design method to a full, 30-period photonic structure. The primary goal is to optimize the geometry of all pillars simultaneously to produce an accelerating field (`e1`) that matches a predefined target curve—in this case, a cosine-shaped field profile.

## Key Objectives
- **Full-Structure Optimization**: To scale the optimization from a single period to a 30-period structure, accounting for interactions between adjacent cells.
- **Custom ADAM Optimizer**: To implement a custom ADAM gradient descent algorithm to efficiently navigate the high-dimensional parameter space.
- **Basin-Hopping Strategy**: To use a basin-hopping approach, running the optimizer from multiple random starting points to find a globally competitive solution and avoid poor local minima.
- **Advanced Constraints**: To apply complex constraints that ensure the final design is physically realistic, with smooth transitions between neighboring pillar geometries.

## Methodology
The notebook defines a cosine wave as the target accelerating field (`d_n`). It then runs the custom ADAM optimizer for multiple iterations from ten different random starting configurations. The cost function is the mean squared error between the simulated field and the target curve. The final output is the set of pillar parameters that yields the best fit to the design curve across all optimization runs.

In [ ]:
import sys, os
sys.path.append("C:\\Program Files\\Lumerical\\v212\\api\\python\\") #Default windows lumapi path
import numpy as np
import matplotlib.pyplot as plt
import lumapi
from scipy.fft import fft, ifft, fftfreq, fftshift
import scipy.optimize as so
import scipy.integrate as integrate
from scipy.optimize import curve_fit
from numpy import unravel_index
from numpy import linalg as npla
from scipy.optimize import basinhopping
import random
from IPython.core.debugger import set_trace
import math
import time
import cmath
from scipy import interpolate
from scipy import signal

In [2]:
lml = 1.932e-6 #laser wavelength
beta = 0.4 #electron velocity 
Periods = 30

x_adj_arr = 0.0
z_adj_arr = 1.5e-6                            
lambda0 = lml
c = 3e8 #speed of light 
f = c/lambda0 #frequency
q = 1 #charge in e
m = 0.511e6 #mass in eV/c**2
Ey = 1 

beta_array = np.full((Periods,), beta) #array of electron velocity at each period
Periods_array = lml*beta_array #periodicity of each cell
preperarr = Periods_array.copy()
delta_W = np.zeros((Periods,1))
EyFmax_forward = np.ones((Periods), dtype=complex)
prev_G = np.full((Periods,1), 100.0)

'''
#exponential design curve
a = np.linspace(np.log(20), np.log(80), Periods)
z = np.exp(a)
d_n = np.reshape(z,(Periods,1)) 
'''

a = np.linspace(0,2*np.pi,Periods)
z = (np.cos(a) + 1)/12
d_n =  np.reshape(z,(Periods,1)) 

#2*(a/(2*np.pi) - (1/2 + a/(2*np.pi)))




max_Periods_array = Periods_array.copy()

#variable declarations
Adjoint_phase = 0.0
lockin = 15 #optimizes the structure from begining to end, 'locking in' the values below a tolerance
lockin_prev = 15
x0point = 0
tol = 0.06 #tolerance parameter for the optimizer, difference of e1 and d_n for ~3 periods
e = []
rarr = []
ephi = []
lmp = lml*beta #synchronicity condition for m=1
i = 0 #used for numbering files
number_of_params = 3

d_n

array([[0.16666667],
       [0.16471838],
       [0.15896462],
       [0.14967442],
       [0.13728219],
       [0.12236737],
       [0.10562736],
       [0.08784491],
       [0.0698515 ],
       [0.05248849],
       [0.03656774],
       [0.02283371],
       [0.01192857],
       [0.00436224],
       [0.0004885 ],
       [0.0004885 ],
       [0.00436224],
       [0.01192857],
       [0.02283371],
       [0.03656774],
       [0.05248849],
       [0.0698515 ],
       [0.08784491],
       [0.10562736],
       [0.12236737],
       [0.13728219],
       [0.14967442],
       [0.15896462],
       [0.16471838],
       [0.16666667]])

In [3]:
def e1_scaling_sigmoid(desired_beta):
    energy_scaling = 0.1681616 + 0.71904549/(1+np.exp(-15.87927686*(desired_beta-0.48068445)))
    return energy_scaling *0.75

Barray = np.sqrt((2*d_n)/m)
d_n = e1_scaling_sigmoid(Barray)

In [3]:
d_n

array([[0.16666667],
       [0.16471838],
       [0.15896462],
       [0.14967442],
       [0.13728219],
       [0.12236737],
       [0.10562736],
       [0.08784491],
       [0.0698515 ],
       [0.05248849],
       [0.03656774],
       [0.02283371],
       [0.01192857],
       [0.00436224],
       [0.0004885 ],
       [0.0004885 ],
       [0.00436224],
       [0.01192857],
       [0.02283371],
       [0.03656774],
       [0.05248849],
       [0.0698515 ],
       [0.08784491],
       [0.10562736],
       [0.12236737],
       [0.13728219],
       [0.14967442],
       [0.15896462],
       [0.16471838],
       [0.16666667]])

In [4]:
W_array = np.full(Periods, m/2*beta**2) #injection energy

def accel_calc(period, EY):
    '''

    Takes in a periodicity array and array of e1 values and calculates energy and velocity and then 
    returns an updated periodicity array.  
    
    '''
    global delta_W
    global beta_array
    global W_array
    for j in range(Periods-1):
        #energy calculation from 6D tracking eq 7
        delta_W[j] = q*period[j]*np.real(EY[j]*np.exp(2*np.pi*1j*period[j]/max_Periods_array[j]))*100e6 #possibly need to add a phase term here   
        W_array[j+1] = W_array[j] + delta_W[j]
        #beta calculated from energy
        beta_array[j+1] =  np.sqrt((2*W_array[j+1])/m)
        period[j+1] = lml*beta_array[j+1]
    #update the last element of the array
    delta_W[-1] = q*period[-1]*np.real(EY[-1]*np.exp(2*np.pi*1j*period[-1]/max_Periods_array[-1]))*100e6
    return period

In [5]:
#max_Periods_array = accel_calc(max_Periods_array, d_n.flatten()) #calculate the max_Periods from the design curve
FDTDxspan = 4.2e-6
FDTDyspan = np.sum(max_Periods_array)
FDTDdx = 10.0e-9
FDTDdy = 10.0e-9
yarrsize = math.ceil(max_Periods_array[-1]/FDTDdy) #max array size to interpolate all cells to, for numpy array operations
xarrsize = math.ceil(FDTDxspan/(2*FDTDdx)) + 1
cell_size = yarrsize//Periods
Xmax = np.linspace(0.0, FDTDxspan/2, xarrsize)
Ymax = np.linspace(0.0, FDTDyspan, yarrsize)
max_Periods_array

array([7.728e-07, 7.728e-07, 7.728e-07, 7.728e-07, 7.728e-07, 7.728e-07,
       7.728e-07, 7.728e-07, 7.728e-07, 7.728e-07, 7.728e-07, 7.728e-07,
       7.728e-07, 7.728e-07, 7.728e-07, 7.728e-07, 7.728e-07, 7.728e-07,
       7.728e-07, 7.728e-07, 7.728e-07, 7.728e-07, 7.728e-07, 7.728e-07,
       7.728e-07, 7.728e-07, 7.728e-07, 7.728e-07, 7.728e-07, 7.728e-07])

In [6]:
W_array[-1]/W_array[0]

1.0

In [7]:
rguess1 = None
rguess1 = np.full((Periods, number_of_params), [180e-9, 180e-9, lmp/2])
for j in range(Periods):
    rguess1[j][2] =  (0.5*max_Periods_array[j])+np.sum(max_Periods_array[:j])
#rguess1 = np.append(rguess1,np.reshape(Periods_array.copy(), (Periods,1)),axis=1)
rguess1 = rguess1.flatten()

In [8]:
def adjoint_source_calc(periods):
    
    '''
    This function calculates the phase of the adjoint source for each cell's given periodicity. 
    It makes an Ey array and 2 y arrays. The purpose of the two y-arrays is one is used to calculate 
    the phase of Ey (using the cell's periodicity), while the second is returned as the y position 
    array for the adjoint source in lumerical. Conceptually it creates multiple sinusoids of increasing
    frequency (for the increasing period size) and then concatenates them. 
    
    It takes in a periodicity array and returns Ey and the concatenated y array. 
    '''

    Ey_n = np.zeros((Periods,yarrsize), dtype='complex')
    y_adj_arr = np.zeros((Periods,yarrsize))
    y_adj_arr2 = np.zeros((Periods,yarrsize))
    y_adj_arr[0] = np.linspace(0.0, periods[0], yarrsize)
    y_adj_arr2[0] = np.linspace(0.0, periods[0], yarrsize)
    
    for j in range(1,Periods):
        y_adj_arr[j] = np.linspace(j*periods[j], (j+1)*periods[j], yarrsize) #array for the cell size for the phase calculation 
        y_adj_arr2[j] = np.linspace(np.sum(periods[:j]), np.sum(periods[:j+1]), yarrsize)  #array for the ypos of the Ey points
        
    for j in range(Periods):
        Ey_n[j] = 1j*np.exp(1j*2*np.pi*(y_adj_arr[j]+periods[j]/2)/periods[j]) #Ey phase calculation
        
        
    Ey = np.ravel(Ey_n) #flattens the arrays before returning
    y_adj_arr = np.ravel(y_adj_arr2)
    
    return Ey, y_adj_arr

In [9]:
Ey, y_adj_arr = adjoint_source_calc(max_Periods_array)

In [10]:
def open_lumerical():
    ''' 
    
    This function opens lumerical via lumapi and sets up an FDTD simulation area.
    
    '''
    
    global fdtd 
    fdtd = lumapi.FDTD()
    
    fdtd.addfdtd()
    fdtd.redrawoff() #just turns the animation off to save simulation time
    #self explanatory settings
    configuration = (
        
        ("FDTD", (("dimension", "2D"), 
                    ("y", FDTDyspan/2), 
                    ("y span", (FDTDyspan)), #extend the FDTD region 3 wavelengths in each direction 
                                                    #to elminate 'ringing' from TFSF source
                    ("x", 0.0), 
                    ("x span", FDTDxspan), 
                    ("z", 1.5e-6), 
                    ("simulation time", 130e-15), 
                    ("mesh type", "uniform"),
                    ("dx", FDTDdx), 
                    ("dy", FDTDdy), 
                    ("y min bc", "Periodic"), 
                    ("y max bc", "Periodic"),
                    ("x min bc", "Symmetric") 
                    )),
                
        
        
    )
    for obj, parameters in configuration:
        for k, v in parameters:
            fdtd.setnamed(obj, k, v)
            
            
    return 

In [11]:
def monitors_func(r):
    '''
    
    This function sets up the field and index monitors for each simulation, taking in the periodicity array as a variable.
    
    '''
    period = r.copy()
    for x in range(Periods):
        fdtd.addindex()
        fdtd.set("name", "index_monitor-{}".format(x))
        fdtd.addpower()
        fdtd.set("name","field_profile-{}".format(x))
        configuration = (
        ("field_profile-{}".format(x), (("monitor type",7),  # 2D z-normal
                    ("spatial interpolation", "none"),
                    ("apodization", "Full"), # temporal apodization of the field monitor before the fourier transform
                    ("apodization center", 50e-15), # used to reduce transient source interference from edge effects
                    ("apodization time width", 30e-15), #set up after viewing the field with a time monitor to balance 
                                                        #information retention and transient source reduction
                    ("y", (0.5*period[x])+np.sum(period[:x])),
                    ("y span", period[x]),
                    ("x",0.0),
                    ("x span", FDTDxspan),
                    ("z", 1.5e-6))), 
            
        ("index_monitor-{}".format(x), (('override global monitor settings', True),
                    ("monitor type", '2D Z-normal'),
                    ('frequency points', 1),
                    ('record conformal mesh when possible', True),
                    ("spatial interpolation", "none"),
                    ("y", (0.5*period[x])+np.sum(period[:x])), # monitors set at y given by the sum of previous periods' size
                    ("y span", period[x]),
                    ("x", 0.0),
                    ("x span", FDTDxspan),
                    ("z", 1.5e-6))),    
            
            
            
        )
    
        for obj, parameters in configuration:
            for k, v in parameters:
                fdtd.setnamed(obj, k, v)
                
    return

In [12]:
def build_structure(r):
    '''
    
    This function builds the dual pillar structure according to the parameter input array r.
    
    
    '''
    for j in range(Periods):
        rx=r[j][0]
        ry=r[j][1]
        pillary = r[j][2]
        #theta1 = r[j][3]*1e8  #### should be able to easily add pillar angle as a parameter
        # when a parameter is added to r, you have to be careful with the constraints, they will likely have to change
        # from every 3rd element to every 4th element 
        channelw = 225.0e-9 #channel width 
        pillarx = (2*rx + channelw)/2 #always maintains the channel width based on rx
        fdtd.addcircle()
        fdtd.set("name", "pillar-R-{}".format(j))
        pillconfiguration = (
        ("pillar-R-{}".format(j), (("radius", rx),
                ("y", pillary),
                ("x", pillarx),
                ("z", 1.5e-6),
                ("z span", 3.0e-6),
                ("make ellipsoid", 1),  #select checkbox 
                ("radius 2", ry),
                ("first axis", "z"),
                #("rotation 1", theta1),
                ("material","Si (Silicon) - Palik"))), 
        )
            
        for obj, parameters in pillconfiguration:
            for k, v in parameters:
                fdtd.setnamed(obj, k, v)
    
    return

In [13]:
def delete_structure():
    '''
    
    This function deletes the structure, field, and index monitors. 
    
    '''
    for j in range(Periods):
        fdtd.select("pillar-R-{}".format(j))
        fdtd.delete()
        fdtd.select("field_profile-{}".format(j))
        fdtd.delete()
        fdtd.select("index_monitor-{}".format(j))
        fdtd.delete()
    fdtd.unselectall()
    return 

In [14]:
def forward_sim(): #make forward sim setup
    
    '''
    
    This function adds the forward source, saves the lsf file, runs the simulation, retrieves the data, does some computation
    and then returns the data. 
    
    '''
    
    
    fdtd.addplane()
    fdtd.set("name", "source_1")
    configuration = (    
        ("source_1", (("amplitude", 1.0),
                    ("phase", 0.0),
                    ("injection axis", "x"),
                    ("direction", "Backward"),
                    ("y", FDTDyspan/2),
                    ("y span", (FDTDyspan)),
                    ("x", FDTDxspan/2),
                    ("z", 1.5e-6),
                    ("z span", 3.0e-6),
                    ("center wavelength", 1.932e-6),
                    ("wavelength span", 0.0))),
        )
    for obj, parameters in configuration:
        for k, v in parameters:
            fdtd.setnamed(obj, k, v)
    
    '''
    
    
    fdtd.addtfsf()
    fdtd.set("name", "source_1")
    configuration = (    
        ("source_1", (("amplitude", 1.0),
                    ("phase", 0.0),
                    ("injection axis", "x"),
                    ("direction", "Backward"),
                    ("y", FDTDyspan/2),
                    ("y span", (FDTDyspan + 6*lml - 10e-9)),
                    ("x", 0.0),
                    ("x span", (FDTDxspan)), 
                    ("z", 1.5e-6),
                    ("z span", 3.0e-6),
                    ("center wavelength", 1.932e-6),
                    ("wavelength span", 0.0))),
        )
    for obj, parameters in configuration:
        for k, v in parameters:
            fdtd.setnamed(obj, k, v)
    '''        
            
                  
    fdtd.save("ForwardSimTest-1.fsp")
    fdtd.run()
    #set_trace()
    EYsize = fdtd.getdata("field_profile-0", "Ey") #array [xvals, yvals, z, lambda]
    x0point = EYsize.shape[0]//2
    EYarrForward = np.zeros((Periods, xarrsize, yarrsize), dtype = complex) #array [Period, xval, yval]
    YarrForward = np.zeros((Periods, yarrsize))  
    XarrForward = np.zeros((Periods, xarrsize))           
    
    for x in range(Periods):
        EYholder = fdtd.getdata("field_profile-{}".format(x), "Ey")
        Yholder = fdtd.getdata("field_profile-{}".format(x), "y")
        Xholder = fdtd.getdata("field_profile-{}".format(x), "x")
        Xmax = np.linspace(Xholder[x0point,0], Xholder[-1,0], xarrsize) #interpolates x to xarrsize
        Ymax = np.linspace(Yholder[0,0], Yholder[-1,0], yarrsize) #interpolates y to yarrsize
        X, Y = Xholder[x0point:,0], Yholder[:,0] 
        YarrForward[x] = Ymax
        XarrForward[x] = Xmax
        EY_real = np.real(EYholder[x0point:,:,0,0])
        EY_imag = np.imag(EYholder[x0point:,:,0,0])
        ip2d_real = interpolate.RectBivariateSpline(X, Y, EY_real) #interpolates Re{Ey}
        ip2d_imag = interpolate.RectBivariateSpline(X, Y, EY_imag) #interpolates Im{Ey} 
        EY_real2 = ip2d_real(Xmax, Ymax)
        EY_imag2 = ip2d_imag(Xmax, Ymax)
        EYarrForward[x] = EY_real2 + 1j*EY_imag2 #recombines the interpolated Ey
        
    fdtd.switchtolayout()
    fdtd.select("source_1")
    fdtd.delete()
    fdtd.unselectall()

    return EYarrForward, YarrForward, XarrForward, x0point 

In [15]:
def adjoint_sim():
    '''
    
    This function sets up and adds the adjoint source, saves the lsf file, runs the simulation, retrieves the data, 
    does some computation and then returns the data. 
    
    '''
    #have to add custom plane wave source with different params to mimic the electron beam/dipole 
    Ex = Ey*0.0
    Ez = Ey*0.0
    
    
    fdtd.addimportedsource()
    fdtd.set('name', 'source_2')
    
    fdtd.putv('x',x_adj_arr)
    fdtd.putv('y',y_adj_arr)
    fdtd.putv('z',z_adj_arr)
    fdtd.putv('Ex',Ex)
    fdtd.putv('Ey',Ey)
    fdtd.putv('Ez',Ez)
    fdtd.putv('f',f)
    fdtd.putv('c',c)
    fdtd.eval('EM = rectilineardataset("EM fields",x,y,z); EM.addparameter("lambda",c/f,"f",f); EM.addattribute("E",Ex,Ey,Ez); select("source_2"); importdataset(EM);')
    
    configuration = (    
            ("source_2", (("amplitude", 1.0),
                        ("phase", Adjoint_phase),
                        ("injection axis", "x"),
                        ("direction", "Forward"),
                        ("y", FDTDyspan/2),
                        ("x", 0.0), 
                        ("z", 1.5e-6),
                        ("center wavelength", 1.932e-6),
                        ("wavelength span", 0.0))),
            )
    for obj, parameters in configuration:
        for k, v in parameters:
            fdtd.setnamed(obj, k, v)
    
    print(Adjoint_phase)
    
    fdtd.save("AdjointSimTest-1.fsp")
    fdtd.run()
    EYsize = fdtd.getdata("field_profile-0", "Ey") #array [xvals, yvals, z, lambda]
    x0point = EYsize.shape[0]//2
    EYarrAdj = np.zeros((Periods, xarrsize, yarrsize), dtype = complex)
    YarrAdj = np.zeros((Periods, yarrsize))  
    for x in range(Periods):
        EYholder = fdtd.getdata("field_profile-{}".format(x), "Ey")
        Yholder = fdtd.getdata("field_profile-{}".format(x), "y")
        Xholder = fdtd.getdata("field_profile-{}".format(x), "x")
        Xmax = np.linspace(Xholder[x0point,0], Xholder[-1,0], xarrsize) #interpolates x to xarrsize
        Ymax = np.linspace(Yholder[0,0], Yholder[-1,0], yarrsize) #interpolates y to yarrsize
        X, Y = Xholder[x0point:,0], Yholder[:,0]
        YarrAdj[x] = Ymax
        EY_real = np.real(EYholder[x0point:,:,0,0])
        EY_imag = np.imag(EYholder[x0point:,:,0,0])
        ip2d_real = interpolate.RectBivariateSpline(X, Y, EY_real) #interpolates Re{Ey}
        ip2d_imag = interpolate.RectBivariateSpline(X, Y, EY_imag) #interpolates Im{Ey}   
        EY_real2 = ip2d_real(Xmax, Ymax)
        EY_imag2 = ip2d_imag(Xmax, Ymax)
        EYarrAdj[x] = EY_real2 + 1j*EY_imag2 #recombines Ey
        
    fdtd.switchtolayout()
    fdtd.select("source_2")
    fdtd.delete()
    fdtd.unselectall()    
    
    
        
    return EYarrAdj, YarrAdj #[xvals, yvals] [ypos]

In [16]:
def FourierFunc(Ey):
    '''
    
    This function performs the fourier transform on the fields from time domain to spatial. It takes in a complex Ey for
    a given cell and does the fft and selects the first non-DC harmonic. 
    
    '''
    
    EyF = fft(Ey) #scipy fft
    N = yarrsize #size of Ey
    EyFm = (2.0/N * EyF) #normalization of Ey given the size
    
    return EyFm[1] # first spatial harmonic

In [17]:
def get_eps_matrix():
    '''
    
    This function retrieves and returns the complex refractive index from the index monitors. 
    
    '''
    full_eps_matrix = np.zeros((Periods, xarrsize, yarrsize), dtype = complex)
    for j in range(Periods):
        ntest = fdtd.getresult("index_monitor-{}".format(j),"index")
        ny = np.squeeze(np.squeeze(np.array(ntest["index_y"][x0point:,:]), axis=2), axis=2)
        X, Y = ntest['x'][x0point:,0], ntest['y'][:,0]
        Xmax = np.linspace(ntest['x'][x0point,0], ntest['x'][-1,0], xarrsize) #interpolates x to xarrsize
        Ymax = np.linspace(ntest['y'][0,0], ntest['y'][-1,0], yarrsize)  #interpolates y to yarrsize
        n_real = np.real(ny)
        n_imag = np.imag(ny)
        ip2d_real = interpolate.RectBivariateSpline(X, Y, n_real) #interpolates Re{n}
        ip2d_imag = interpolate.RectBivariateSpline(X, Y, n_imag) #interpolates Im{n}
        n_real2 = ip2d_real(Xmax, Ymax)
        n_imag2 = ip2d_imag(Xmax, Ymax)
        full_eps_matrix[j] = n_real2 + 1j*n_imag2 #recombines complex refractive index
        
    return full_eps_matrix  

In [18]:
def conversion_func(j, y_pos, prev_period):
    '''
    I was using this to keep relative ypos when periodicity was a dynamic parameter.
    '''
    y_rel = (y_pos - np.sum(prev_period[:j]))
    y_ratio = y_rel/prev_period[j]
    return y_ratio

In [19]:
def stepping_func(k, R, Gw):
    '''
    
    This function is what increments the parameters before taking delta epsilon. Becasuse of how the gradient is calculated
    all constraints, weights, or modifications to the gradient for optimization purposes has to be done here. Since the gradient
    is calculated for the entire structure and there are interactions between the cells, the gradient cannot be modified after
    it has been calculated without losing information. 
    
    Takes parameter number (k), parameter array (R), and weights array (Gw) as arguments and returns an updated, constrained
    bounded, weighted, paramter array (R).
    
    '''
    
    
    rng=20e-9
    
    
    if k == 0:
        for j in range(Periods):
            #constrains rx
            constrained_rng = constraint_fun(x_cons_var, x_cons_val, len(x_cons_var)//Periods, j, k, R.flatten(), rng)
            constrained_rng *= Gw[j]*np.abs(gradam[j,k])
            R[j,k] = R[j,k] + constrained_rng
    elif k == 1:
        for j in range(Periods):
            #constrains ry
            constrained_rng = constraint_fun(y_cons_var, y_cons_val, len(y_cons_var)//Periods, j, k, R.flatten(), rng)
            constrained_rng *= Gw[j]*np.abs(gradam[j,k])
            R[j,k] = R[j,k] + constrained_rng
    elif k == 2:
        for j in range(Periods):
            #constrains ypos 
            constrained_rng = constraint_fun2(ypos_cons_var, ypos_cons_val, len(ypos_cons_var)//Periods, j, k, R.flatten(), rng)
            constrained_rng *= Gw[j]*np.abs(gradam[j,k])
            R[j,k] = R[j,k] + constrained_rng
    '''        
    elif k == 3:
        # was used to maintain the relative y_pos when the period size was incremented
        prev_period_array = R[:,3].copy()
        for j in range(lockin-3,Periods):
            R[j][k] = R[j][k] + rng
            y_ratio = conversion_func(j, R[j][2], max_Periods_array) #maintains the realtive position of ypos
            R[j,2] = R[j,3]*y_ratio + np.sum(R[:j,3])
    '''               
                
    lobounds = bnds[0]
    for j in range(Periods):
        lobounds[j][2] = np.sum(max_Periods_array[:j]) + R[j][1]

    hibounds = bnds[1]
    for j in range(Periods):
        hibounds[j][2] = np.sum(max_Periods_array[:j+1]) - R[j][1]
        

    R1 = R.flatten()

    lobounds = lobounds.flatten()
    hibounds = hibounds.flatten()
    #apply hard boundaries to parameter values
    for z in range(len(R1)):
        if R1[z] > hibounds[z]:
            R1[z] = hibounds[z]
        elif R1[z] < lobounds[z]:
            R1[z] = lobounds[z]
        
    R = np.reshape(R1, (Periods,number_of_params))        
            
    return R

In [20]:
def delta_eps_phi(eps_phi, R, eps0):
    
    '''
    
    This function calculates delta epsilon/delta phi from Manuel thesis eq. 15
    
    Takes epsilon matrix, parameter array, and epsilon_0 matrix as arguments and returns an updated epsilon matrix
    and a copy of the initial parameters array. 
    
    '''
    # have to set multiple r's because I was having issues with python deep copies, I wanted 2 arrays, one that would be 
    # incremented and updated and a copy of the intitial so that each parameter could be incremented individually and reset
    # so as to take the gradient wrt each parameter....
    r1 = R.copy()
    r2 = r1.copy()
    rguesscopy = r1.copy()
    
    # calculate weights based on tolerance paramter
    Gweights = mappingfunc(prev_G)
    Gweights = np.reshape(Gweights,(Periods,1))

    for k in range(number_of_params):
        r1 = stepping_func(k, r2, Gweights)
        build_structure(r1) # build structure and get eps 
        monitors_func(max_Periods_array)
        eps_phi[k] = get_eps_matrix() 
        eps_phi[k] = eps_phi[k] - eps0  #deltaepsilon/delta phi
        r2 = rguesscopy.copy()   # reset to original params (makes a copy of r guess to go back to each time)
        delete_structure()
        
    return eps_phi, rguesscopy

In [21]:
def calcs(deps_dphi_n, EyF, EyA, yvals, xvals):
    '''
    
    This function calculates the gradient of each period dG/dphi eq. 18 in Manuel. 

    Takes deps_dphi, e1 values, adjoint fields, y values and x values and returns d_G/d_phi_n. 
    
    
    '''

    deps_dphi_n_copy = deps_dphi_n.copy()
    Eproduct = np.zeros_like(EyA, dtype=complex)
    for j in range(Periods):
        Eproduct[j] = EyF[j]*EyA[j] #e1 values * Adjoint field
    constants = -2*(2*np.pi/lml)**2 
    for j in range(Periods):
        deps_dphi_n_copy[:,j,:,:] = deps_dphi_n[:,j,:,:]*Eproduct[j]
    dG_dphi_n = np.zeros((number_of_params, Periods), dtype='complex')
    for l in range(number_of_params):
        for m in range(Periods):
            int_2d = np.trapz(np.trapz(deps_dphi_n_copy[l,m,:,:], xvals[m], axis=0), yvals[m], axis=0) #integrates the fields
            #over the length of the structure
            dG_dphi_n[l,m] = int_2d #labeled param num, Period num, x,y values have been integrated out 
    dG_dphi_n = dG_dphi_n
    
    dG_dphi_n_reshape = np.zeros((Periods,number_of_params)) #reshapes to [Periods, params]
    for j in range(dG_dphi_n.shape[0]):
        for k in range(dG_dphi_n.shape[1]):
            dG_dphi_n_reshape[k,j] = np.real(dG_dphi_n[j,k])
    
    dG_dphi_n_reshape = dG_dphi_n_reshape.copy()*constants 
    
    return dG_dphi_n_reshape

In [22]:
def func(rmin):
    '''
    
    This is the main function which is run by the optimizer. It takes the flattened parameter array as its input and 
    returns deltaG and dG_dphi to calculate the gradient. It opens and closes a new lumerical window every time. It runs
    the forward simulation, adjoint simulation, gathers the epsilon matrix and calculates dG_dphi. 
    
    '''
    rguess = np.reshape(rmin.copy(), (Periods,number_of_params)) #Periods x Params initial value array
    rarr.append(rguess)
    global preperarr
    global FDTDyspan
    global Periods_array
    preperarr = Periods_array.copy()
    FDTDyspan = np.sum(max_Periods_array)
    
    open_lumerical()    
    build_structure(rguess)   
    monitors_func(max_Periods_array)
    
    global i
    i+=1
    
    EyFor, Ly, Lx, x0point = forward_sim() #Ey values, yvalues, xvalues, and the middle of the x array due to symmetric boundary

    EyFmax_forward = np.ones((Periods), dtype=complex) 
    for j in range(Periods):
        EyFmax_forward[j] = FourierFunc(EyFor[j,0,:])
    EyFmax_forward = EyFmax_forward/2 #divide by 2 because of symmetric boundary cond.  
    
    #accel_calc(max_Periods_array, EyFmax_forward)
    #Periods_array = lml*beta_array
    
    global Adjoint_phase 
    #if lockin > lockin_prev:
    Adjoint_phase = - np.angle(EyFmax_forward[0], deg=True) 
        
    #global Ey, y_adj_arr
    #Ey, y_adj_arr = adjoint_source_calc(max_Periods_array) this is only needed if Periods_array becomes a dynamic variable again
    EyAdj, yAdj = adjoint_sim()

    eps0 = get_eps_matrix()
    delete_structure()
    eps_phi = np.zeros((number_of_params, Periods, xarrsize, yarrsize), dtype='complex') 
    deps_dphi_n = np.zeros((number_of_params, Periods, xarrsize, yarrsize), dtype='complex') #param number, period number, lx, ly    
    deps_dphi, rguesscopy = delta_eps_phi(eps_phi, rguess, eps0)

    dG_dphi_n = calcs(deps_dphi, EyFmax_forward, EyAdj, Ly, Lx)
    
    #G_n = delta_W.copy() if its decided to use energy change as the design curve/cost function

    G_n = np.real(EyFmax_forward.copy()*np.exp(-1j * np.angle(EyFmax_forward[0])))
    G_n = np.reshape(G_n,(Periods,1))
    kernel_size = 3
    kernel = np.ones(kernel_size) / kernel_size
    G_conv = np.convolve(G_n.flatten(), kernel, mode='same')
    G_conv[0] = np.sum(G_n[:2])/2
    G_conv[-1] = np.sum(G_n[-2:])/2
    G_n = np.reshape(G_conv,(Periods,1)) #smoothes e1 values with a rolling mean of 3 (2 at boundaries) to reduce transient 
                                        # source interference

    eholder = G_n.flatten() #cant append unless its a flat array
    deltaGd = G_n-d_n
    #deltaGd[:15] = 0.0
    #deltaGd[-15:] = 0.0
    deltaGd = - np.reshape(deltaGd,(Periods,1)) #allows matrix manipulations rather than having to loop
    e.append(eholder)
    #print(G_n, dG_dphi_n)
    fdtd.deleteall()
    np.savetxt("final-multi-pillar-optimization.txt", np.array(e))
    #save rarr?
    
    return deltaGd, dG_dphi_n , eholder

In [23]:
deltaRx = 20e-9
deltaRy = 20e-9
RxMax = 270e-9
RxMin = 130e-9
RyMax = 270e-9
RyMin = 130e-9

deltayMin = lambda j=j: (0.25*Periods_array[j])+np.sum(Periods_array[:j])
deltayMax = lambda j=j: (0.75*Periods_array[j])+np.sum(Periods_array[:j])

In [24]:
def LEpiecewise(con_var, con_val, num_of_cons): #returns a weight based on whether a value is <= a constraint 
    return (1)/(1+np.exp(-50*(con_var/con_val - 1)))

def GEpiecewise(con_var, con_val, num_of_cons): #returns a weight based on whether a value is >= a constraint 
    return (1)/(1+np.exp(-50*(con_val/con_var - 1)))
    
    
def NormalizeData(data): #function that normalizes data, not used anymore
    return (data - np.min(data) + 1e-10) / (np.max(data) - np.min(data))

def constraint_fun(g_cons_var, g_cons_val, num_of_cons, idx0, idx1, params, grad):
    '''
    Applies constraints to rx and ry values. 
    
    '''
    cons_holder = np.zeros(num_of_cons)
    for x in range(num_of_cons):
        if g_cons_var[idx0+x*Periods]['type'] == 'less':
            cons_holder[x] = grad*(LEpiecewise(g_cons_var[idx0+x*Periods]['fun'](params), g_cons_val[idx0+x*Periods], num_of_cons))
        else:
            cons_holder[x] = grad*(GEpiecewise(g_cons_var[idx0+x*Periods]['fun'](params), g_cons_val[idx0+x*Periods], num_of_cons))
    # if multiple constraints are violated, the grad is set to zero (no negative/backwards steps)
    if (np.sum(np.abs(cons_holder))) >= (np.abs(grad)): 
        grad = 0.0
    else:
        grad -= np.sum(np.abs(cons_holder))
    
    
    return grad

def constraint_fun2(g_cons_var, g_cons_val, num_of_cons, idx0, idx1, params, grad):
    '''
    Applies constraints to y_pos values. 
    
    '''
    cons_holder = np.zeros(num_of_cons)
    for x in range(num_of_cons):
        if g_cons_var[idx0+x*Periods]['type'] == 'less':
            cons_holder[x] = grad*(LEpiecewise(g_cons_var[idx0+x*Periods]['fun'](params), g_cons_val[idx0+x*Periods](params), num_of_cons))
        else:
            cons_holder[x] = grad*(GEpiecewise(g_cons_var[idx0+x*Periods]['fun'](params), g_cons_val[idx0+x*Periods](params), num_of_cons))
    # if multiple constraints are violated, the grad is set to zero (no negative/backwards steps)
    if (np.sum(np.abs(cons_holder))) >= (np.abs(grad)):
        grad = 0.0
    else:
        grad -= np.sum(np.abs(cons_holder))
    
    return grad

In [25]:
def weightingfunc(con_var, con_val):
    '''
    Sets the weights based on tolerance parameter. Below tolerance gets automatically set to 0. 
    '''
    if con_var <= con_val:
        return 0.0
    else:
        return (1)/(1+np.exp(-5*(con_var/con_val - 1)))
    
    
def mappingfunc(Gd):
    
    '''
    Sets the weights based on delta_G values of several periods in order to account for short range interactions.
    '''
    
    w_to_G = np.zeros(Periods)
    
    global lockin
    global lockin_prev
    lockin_prev = lockin
    
    
    G_to_w = np.sum(np.abs(Gd[lockin:lockin+3])) #the delta_G values of 3 neighboring cells is used to compute weights
    w = weightingfunc(G_to_w, tol)

    for j in range(lockin,Periods, 3):
        w_to_G[j-2:j+5] = weightingfunc(np.sum(np.abs(Gd[j:j+3])), tol)
    
    w_to_G[lockin-2:lockin+5] = w #allows greater freedom for cells beyond the 3 neighbors to again account for interactions
    w_to_G[:5] = w_to_G[5]/2 #try to ignore the first few cells because they have strong edge effects
    
    #locks in the parameters if their delta_G values are below a tolerance
    if w == 0.0:
        if np.flatnonzero(w_to_G[lockin_prev:]).size == 0:
            lockin += 3
        else:
            lockin = np.flatnonzero(w_to_G[lockin_prev:])[0] #the first non-zero element in w2G after lockin 
    

    w_prev1 = w 
    #print(w_to_G)
    w_to_G = np.ones(Periods)
           
    return w_to_G

In [26]:
def step_adam(gradient, mopt_old, vopt_old, iteration, beta1, beta2, epsilon=1e-20):
    
    """ Performs one step of adam optimization"""

    mopt = beta1 * mopt_old + (1 - beta1) * gradient
    mopt_t = mopt / (1 - beta1**(iteration + 1))
    vopt = beta2 * vopt_old + (1 - beta2) * (np.square(gradient))
    vopt_t = vopt / (1 - beta2**(iteration + 1))
    grad_adam = mopt_t / (np.sqrt(vopt_t) + epsilon)

    return (grad_adam, mopt, vopt)

In [27]:
def adam_optimize(objective, parameters, bounds, step_size=20.0e-9, Nsteps=50, direction='min'):
    '''
    
    Performs Nsteps steps of optimization of function `objective` with gradient `jac`.
    
    '''
    paramsarray = []
    params1 = np.array(parameters.copy())
    best_G = 1e6
    best_params = np.full_like(params1, 0.0)
    best_G_n = np.full((Periods), 1e6)
    prev_Gw = 1.0
    wprev = 1.0
    global iteration
    global gradam 
    global mopt
    global vopt
    beta1=0.9 
    beta2=0.999
    gradam = np.ones((Periods,number_of_params))

    for iteration in range(Nsteps):


        deltaGd, grad, G_n = objective(params1)
        
        dG_dphi = 2*deltaGd*grad  #eq 20 from manuel
        G = np.sum(deltaGd**2) #eq 7 from manuel
        grad = np.reshape(dG_dphi.copy(), (Periods, number_of_params))
        params2 = np.reshape(params1.copy(), (Periods,number_of_params))
        paramsarray.append(params2)
        
        if G <= best_G:
            best_G = G.copy()
            best_params = params1.copy()
            best_G_n = G_n.copy()
            
    
        if iteration == 0:
            mopt = 0.0
            vopt = 0.0
    
    
        step1 = step_size*grad*1000 #an observational value that generally allows the step size to be close to what we want
        
        gradam = step1/step_size
        (gradam, mopt, vopt) = step_adam(gradam, mopt, vopt, iteration, beta1, beta2)
        
        grad = grad.flatten()  

        print(step1, deltaGd)
        
        step1 = step1.flatten()
        
        if direction == 'min':
            params1 -= step1
        elif direction == 'max':
            params1 += step1
        else:
            raise ValueError("The 'direction' parameter should be either 'min' or 'max'")
        
        params1 = np.reshape(params1, (Periods,number_of_params))
        #print(params1)                
        params1 = params1.flatten()
            
    return best_params, best_G, best_G_n

In [28]:
num_x_cons = 3 #number of x constraints
num_y_cons = 3 #number of y constraints
num_ypos_con = 2 #number of ypos constraints
x_cons_var = []    
x_cons_val = []
y_cons_var = []    
y_cons_val = []
ypos_cons_var = []
ypos_cons_val = []
for j in range(Periods-1):    #neighboring Rx differ by less than 20nm
    x_cons_var.append({'type' : 'less', 'fun' : lambda x, j=j: np.abs(x[number_of_params*j+number_of_params] - x[number_of_params*j])}) 
    x_cons_val.append(deltaRx)
x_cons_var.append({'type' : 'less', 'fun' : lambda x, j=j: np.abs(x[number_of_params*(Periods-1)] - x[0])})
x_cons_val.append(deltaRx)

for j in range(Periods-1):   #neighboring Ry differ by less than 20nm
    y_cons_var.append({'type' : 'less', 'fun' : lambda x, j=j: np.abs(x[number_of_params*j+1+number_of_params] - x[number_of_params*j+1])})  
    y_cons_val.append(deltaRy)
y_cons_var.append({'type' : 'less', 'fun' : lambda x, j=j: np.abs(x[number_of_params*(Periods-1)+1] - x[1])})
y_cons_val.append(deltaRy)

for j in range(Periods):    # Rx less than 300nm
    x_cons_var.append({'type' : 'less', 'fun' : lambda x, j=j: x[number_of_params*j]}) 
    x_cons_val.append(RxMax)
    
for j in range(Periods):    # Ry less than 300nm
    y_cons_var.append({'type' : 'less', 'fun' : lambda x, j=j: x[number_of_params*j+1]})
    y_cons_val.append(RyMax)
    
for j in range(Periods):    # Rx greater than 100nm
    x_cons_var.append({'type' : 'greater', 'fun' : lambda x, j=j: x[number_of_params*j]})
    x_cons_val.append(RxMin)
    
for j in range(Periods):    # Ry greater than 100nm
    y_cons_var.append({'type' : 'greater', 'fun' : lambda x, j=j: x[number_of_params*j+1]})
    y_cons_val.append(RyMin)
        
for j in range(Periods):    # ypos greater than lmp dynamic constraint
    ypos_cons_var.append({'type' : 'greater', 'fun' : lambda x, j=j: x[number_of_params*j+2]})
    ypos_cons_val.append(lambda x, j=j: np.sum(max_Periods_array[:j]) + x[number_of_params*j+1])
    
for j in range(Periods):    # ypos less than 2lmp dynamic constraint
    ypos_cons_var.append({'type' : 'less', 'fun' : lambda x, j=j: x[number_of_params*j+2]})
    ypos_cons_val.append(lambda x, j=j: np.sum(max_Periods_array[:j+1]) - x[number_of_params*j+1])


In [29]:
ghb = round((lmp/2*0.75)*1e9, -1)
glb = round((lmp/2*0.25)*1e9, -1)
ghb-glb

190.0

In [30]:
#create arrays for the boundaries 

lobounds = np.full((Periods, number_of_params), [100e-9, 100e-9, lmp/2])
for j in range(Periods):
    lobounds[j][2] = np.sum(Periods_array[:j]) + lobounds[j][2]

hibounds = np.full((Periods, number_of_params), [290e-9, 290-9, lmp/2])
for j in range(Periods):
    hibounds[j][2] = np.sum(Periods_array[:j]) + hibounds[j][2]
    
bnds = np.stack((lobounds, hibounds))

In [31]:
grid_size = int(round((ghb-glb)/10,-1))
grid_size
ghb

290.0

In [31]:
np.array(np.loadtxt("retvalarray_nonperiodic-ramp.txt"))

array([6.27973940e-02, 6.72249322e-02, 9.26410036e-02, 1.81877923e-01,
       2.35167670e-01, 3.12850407e-01, 9.26438151e-01, 1.70337744e-02,
       4.72462657e-01, 1.15457197e+00, 4.88556349e-02, 1.66805841e-03,
       8.60039508e-01, 2.97197572e-02, 3.60997561e-02, 7.03442426e-05])

In [32]:
NPbasinarray = np.array(np.loadtxt("basinarray_multi-optimization.txt"))

In [32]:
optresult = np.array(np.loadtxt("optimizedresult-multipillar.txt"))

In [33]:
optresult, minret, best_Gn = adam_optimize(func, optresult, bnds, step_size=25.0e-9, Nsteps=12, direction='min')

-51.398259407284804
[[ 5.81074117e-10  7.26354299e-10 -7.66512209e-13]
 [ 8.91007369e-10  1.13359572e-09 -1.23434976e-11]
 [ 5.91929869e-10  6.83443709e-10  6.73741835e-12]
 [ 7.45315598e-10  8.01575808e-10 -6.77347256e-12]
 [ 5.36871610e-10  5.52296966e-10 -1.15568666e-11]
 [ 2.40431562e-10  2.54905641e-10 -5.81248580e-13]
 [ 2.07000849e-10  2.21449716e-10 -1.86146061e-11]
 [ 1.80787069e-10  1.96867649e-10 -2.83246683e-12]
 [-1.21332547e-10 -1.31358064e-10  6.79174541e-12]
 [ 4.21548413e-10  4.41659159e-10 -2.15404053e-11]
 [-9.95335690e-13 -1.15231820e-12  8.51591041e-13]
 [ 1.35200286e-10  1.33710234e-10 -9.49301438e-12]
 [-5.54295724e-12 -1.88638786e-12  3.33499840e-12]
 [-4.32519531e-11 -4.01382780e-11  1.65016996e-12]
 [-2.31372586e-11 -1.37657534e-11  3.69128582e-12]
 [-2.97256559e-11 -2.26654775e-11  1.94938447e-12]
 [-4.42356660e-11 -3.45452465e-11  2.02358617e-12]
 [-2.83385259e-11 -2.30830166e-11  1.15477340e-12]
 [-2.56660761e-11 -2.25962062e-11 -1.05966893e-13]
 [-6.300846

-48.28981829020286
[[ 3.21854083e-10  4.44508271e-10 -7.69300296e-13]
 [ 5.76164397e-10  1.19315787e-09 -2.24799973e-11]
 [ 3.25360938e-10  4.14815577e-10  9.01853885e-12]
 [ 7.58878477e-10  8.10168215e-10  2.55966744e-12]
 [ 4.35370937e-10  4.50109711e-10 -9.76938743e-12]
 [-2.60777272e-11 -2.63083326e-11 -3.66835268e-13]
 [ 4.23616467e-10  4.63102755e-10 -5.82272764e-11]
 [ 9.13118511e-11  9.86966530e-11 -2.86836961e-12]
 [-1.51618412e-10 -1.56148870e-10  3.40755451e-12]
 [ 3.53693065e-10  3.76547660e-10 -2.41654247e-11]
 [ 5.62331445e-11  5.52973994e-11 -1.27610882e-13]
 [ 2.81310463e-10  2.85765001e-10 -2.35215655e-11]
 [ 3.49870107e-11  5.33264298e-11  1.18675771e-12]
 [-1.63337969e-11 -1.56868624e-11  6.74722790e-13]
 [ 4.03924776e-12  5.05267291e-12  3.75515105e-12]
 [-2.37941004e-11 -1.91155304e-11  1.20041678e-12]
 [-2.96828931e-11 -2.13974427e-11  1.76183496e-12]
 [-1.01465662e-11 -6.20110603e-12  1.04416551e-12]
 [-5.29115278e-13 -4.95133456e-13 -5.73158698e-15]
 [-1.0926749

-45.43904035912979
[[ 1.77795853e-10  2.99656744e-10  1.98226386e-12]
 [ 3.48655049e-10  1.32150048e-09 -2.02480840e-11]
 [ 2.24873734e-10  3.01501082e-10  5.53727351e-12]
 [ 8.02593116e-10  9.86998421e-10 -5.42342306e-12]
 [ 4.14300426e-10  4.32768155e-10  3.87748322e-12]
 [-5.52586280e-11 -5.78974918e-11 -2.23564218e-12]
 [ 6.74952061e-10  7.10126485e-10 -2.14851294e-11]
 [ 1.26157772e-11  1.38340964e-11 -7.92689958e-13]
 [-1.76193470e-11 -1.81144082e-11  5.22723912e-13]
 [ 1.49508157e-10  1.70060407e-10 -9.74813395e-12]
 [ 1.08121222e-10  1.09308975e-10  2.52702835e-12]
 [ 3.56617955e-10  3.65152511e-10 -3.02337054e-11]
 [ 1.16310575e-10  1.27842973e-10 -1.18065469e-13]
 [ 7.86108732e-12  6.99856910e-12 -3.82616333e-13]
 [ 3.74901941e-12  3.06680737e-14  3.72371688e-12]
 [-2.15918312e-11 -1.82627873e-11  8.11654226e-13]
 [-1.95751631e-11 -1.31470215e-11  1.63688642e-12]
 [-1.34014379e-12  8.68271746e-13  1.02226387e-12]
 [ 5.43217463e-12  5.72506253e-12 -2.10166661e-14]
 [ 5.8611660

In [102]:
paramrange = np.array([16, 18, 20, 22])

In [ ]:
##### run accel calc with e[-1] and max_Periods_array then find 40.88+np.sum(dW)=total energy 

In [115]:
np.random.choice(paramrange, 4, replace=False)


array([18, 22, 16, 20])

In [32]:
basinarray = []
retvalarray = []
best_fit_array = []
paramrange = np.array([16, 18, 20, 22])
# ten random starting points each optimized to local minima or maxiter
for y in range(10):
    Adjoint_phase = 0.0
    lockin = 0
    lockin_prev = 0
    rguess1 = np.full((Periods, 3), [random.randrange(16, 24)*10e-9, random.randrange(16, 24)*10e-9, lmp/2])
    for j in range(Periods):
        rguess1[j][2] = (0.5*max_Periods_array[j])+np.sum(max_Periods_array[:j])
    rguess1 = rguess1.flatten()
    #force minimum step size to 1nm 
    param_basin, maxretval, best_fit = adam_optimize(func, rguess1, bnds, step_size=50.0e-9, Nsteps=4, direction='min')
    basinarray.append(param_basin)
    retvalarray.append(maxretval)
    best_fit_array.append(best_fit)
    NPbasinarray = np.array(basinarray)
    NPretvalarray = np.array(retvalarray)
    NPbest_fit_array = np.array(best_fit_array)
    np.savetxt("basinarray_multi-optimization.txt", np.array(NPbasinarray))
    np.savetxt("retvalarray_multi-optimization.txt", np.array(NPretvalarray))
    np.savetxt("best_fit_array-multipillar.txt", np.array(NPbest_fit_array))
    #save params and maxvalue 

-95.97608355100957
[[ 4.16365565e-09  3.27910873e-09 -3.49752777e-11]
 [ 4.04755082e-09  3.06411093e-09 -1.55245637e-11]
 [ 3.92672889e-09  3.02825974e-09 -4.62902945e-11]
 [ 3.61147379e-09  2.86949307e-09 -4.48964428e-11]
 [ 3.15857795e-09  2.40745827e-09  6.59217111e-11]
 [ 2.83252306e-09  2.15859457e-09  5.10030577e-11]
 [ 2.17228089e-09  1.72508687e-09 -5.56646385e-11]
 [ 1.62815209e-09  1.25764710e-09 -7.85857588e-12]
 [ 1.09683358e-09  8.30647936e-10 -6.66373870e-12]
 [ 5.10336554e-10  4.01342900e-10  4.98047018e-13]
 [ 1.83153347e-11  1.44243361e-11  4.59175915e-14]
 [-4.14073441e-10 -3.13465289e-10  1.73583887e-12]
 [-7.71831572e-10 -5.95230045e-10  2.79769820e-12]
 [-1.00529453e-09 -7.98756312e-10  1.38374925e-11]
 [-1.11495858e-09 -8.49817638e-10 -2.75259948e-11]
 [-1.16031023e-09 -8.84243625e-10 -2.64698872e-12]
 [-9.94319918e-10 -7.89626302e-10  2.34416403e-11]
 [-7.60214195e-10 -5.87218630e-10 -7.18860591e-12]
 [-4.26136263e-10 -3.22719138e-10 -7.77101041e-12]
 [ 8.3553656

-124.47958552178252
[[ 2.55959379e-09  3.22972817e-09  6.22218069e-09]
 [-5.37855470e-09 -7.18418806e-09  6.71371780e-11]
 [ 9.64915554e-09  1.17011946e-08 -3.54767128e-09]
 [-1.46199748e-09 -2.17127613e-09  1.66448018e-10]
 [-1.00135659e-08 -1.54391495e-08  6.05852891e-10]
 [-4.96242855e-09 -7.54587264e-09 -5.45953050e-11]
 [-7.66835005e-09 -2.12946457e-10  3.52746107e-10]
 [-5.91538912e-09 -0.00000000e+00  2.03020627e-11]
 [-4.79866670e-09 -1.33858932e-09 -1.35209459e-09]
 [-5.74892497e-09 -3.76571492e-09 -3.53550409e-09]
 [-2.67900799e-09 -2.63663049e-09 -2.59425185e-09]
 [-1.28187588e-09 -1.28094565e-09 -1.25774319e-09]
 [-4.80444190e-10 -4.79971552e-10 -4.64946284e-10]
 [ 1.02851510e-10  1.02851510e-10  1.02688410e-10]
 [ 3.16120808e-10  3.16120808e-10  3.13498571e-10]
 [ 3.38833852e-10  3.38833852e-10  3.34135733e-10]
 [ 4.69031287e-11  4.69031287e-11  4.67915267e-11]
 [-4.63409218e-10 -4.63409218e-10 -4.59736923e-10]
 [-1.23535550e-09 -1.23390722e-09 -1.23026612e-09]
 [-2.736918

12.650367600388211
[[ 2.20627709e-08  2.82840916e-08  3.36621691e-09]
 [ 1.49080418e-08  2.03742579e-08 -3.21471812e-10]
 [-1.38879889e-08 -1.83019486e-08  9.50820065e-11]
 [-4.74511956e-09 -6.81210703e-09  4.03399427e-10]
 [-1.60135533e-08 -2.14041783e-08  1.17912761e-09]
 [-2.73171411e-09 -3.92746170e-09  5.96916945e-10]
 [-4.20010558e-09 -6.27309781e-09  2.42717310e-10]
 [ 5.35341290e-10  6.09264485e-10  1.48133345e-10]
 [ 4.92173303e-10  3.53270960e-10  1.81616458e-10]
 [ 1.33036265e-10  8.46072997e-11  6.43697240e-11]
 [ 2.37298920e-10  2.75288712e-10  4.15737726e-11]
 [ 1.01953712e-10  1.29927090e-10  4.92183374e-12]
 [ 2.59385523e-11  3.35602418e-11  1.38497354e-12]
 [-2.32458694e-11 -2.98285674e-11  1.01810108e-13]
 [-5.78068461e-11 -7.16128336e-11 -1.08731619e-11]
 [-6.39037806e-11 -8.25906468e-11 -7.78497824e-12]
 [-5.13174466e-11 -6.78862693e-11 -3.25308176e-13]
 [-1.17297940e-11 -1.23135576e-11 -8.17132294e-13]
 [ 3.78698963e-11  4.94732343e-11  3.03721434e-12]
 [ 7.8612152

-132.7200115587764
[[ 8.92327758e-10 -8.04385276e-10  3.98810325e-09]
 [ 7.20934292e-10  1.81014098e-09  3.34715289e-09]
 [ 7.58016500e-10  9.93541711e-10  1.69750300e-11]
 [ 2.72020519e-09  4.64435746e-12  1.25452393e-09]
 [ 7.44946987e-08  1.17563928e-07  3.66499578e-08]
 [ 8.78453416e-08  1.44126480e-07  4.62208800e-09]
 [ 8.06108409e-08  1.20464840e-07 -4.60054514e-09]
 [ 5.69589447e-09  1.06269310e-08 -6.46889006e-09]
 [ 1.20378351e-09  1.92815860e-09 -2.74082439e-10]
 [-2.39910917e-10 -3.18703491e-10 -1.46272066e-10]
 [-1.20344626e-10 -1.07484393e-10 -2.92359827e-13]
 [-4.99369808e-11 -7.16554923e-11 -7.28538771e-12]
 [-2.32804594e-11 -3.19610014e-11 -3.86845109e-12]
 [-4.78979810e-12 -6.39237404e-12 -7.29203773e-13]
 [ 7.06502915e-13 -9.30024722e-13 -1.10321135e-13]
 [-1.22642917e-11 -1.35055931e-11 -2.09973195e-12]
 [-2.95356734e-11 -2.14388882e-11 -4.14307935e-13]
 [-7.53988204e-11 -3.38581229e-11 -2.50912189e-12]
 [-1.40796749e-10 -5.52068148e-11 -1.60800876e-11]
 [ 6.5828810

-95.97608355100957
[[ 4.16365565e-09  3.27910873e-09 -3.49752777e-11]
 [ 4.04755082e-09  3.06411093e-09 -1.55245637e-11]
 [ 3.92672889e-09  3.02825974e-09 -4.62902945e-11]
 [ 3.61147379e-09  2.86949307e-09 -4.48964428e-11]
 [ 3.15857795e-09  2.40745827e-09  6.59217111e-11]
 [ 2.83252306e-09  2.15859457e-09  5.10030577e-11]
 [ 2.17228089e-09  1.72508687e-09 -5.56646385e-11]
 [ 1.62815209e-09  1.25764710e-09 -7.85857588e-12]
 [ 1.09683358e-09  8.30647936e-10 -6.66373870e-12]
 [ 5.10336554e-10  4.01342900e-10  4.98047018e-13]
 [ 1.83153347e-11  1.44243361e-11  4.59175915e-14]
 [-4.14073441e-10 -3.13465289e-10  1.73583887e-12]
 [-7.71831572e-10 -5.95230045e-10  2.79769820e-12]
 [-1.00529453e-09 -7.98756312e-10  1.38374925e-11]
 [-1.11495858e-09 -8.49817638e-10 -2.75259948e-11]
 [-1.16031023e-09 -8.84243625e-10 -2.64698872e-12]
 [-9.94319918e-10 -7.89626302e-10  2.34416403e-11]
 [-7.60214195e-10 -5.87218630e-10 -7.18860591e-12]
 [-4.26136263e-10 -3.22719138e-10 -7.77101041e-12]
 [ 8.3553656

-56.901692139775115
[[ 4.79940958e-09  6.20786932e-09 -6.43595151e-11]
 [ 4.55848475e-09  5.96077770e-09  3.22301615e-12]
 [ 4.55324454e-09  5.85688108e-09 -9.39731585e-12]
 [ 3.64921285e-09  4.68276720e-09  1.86636464e-11]
 [ 2.80451001e-09  3.50631186e-09  1.14798969e-10]
 [ 1.67337419e-09  2.12179612e-09 -1.86690790e-11]
 [ 1.78755119e-10  2.25176456e-10  7.00971434e-12]
 [-8.61797347e-10 -1.07341740e-09  7.37367638e-12]
 [-1.54384748e-09 -2.06714453e-09  5.82768456e-11]
 [-2.10023022e-09 -2.72411034e-09  6.33764306e-11]
 [-2.67149493e-09 -3.42195206e-09  2.54201723e-11]
 [-2.43346390e-09 -3.07859303e-09  7.69275187e-11]
 [-2.13370689e-09 -2.65437503e-09  6.48884749e-11]
 [-2.46256876e-09 -2.98186774e-09  6.97029746e-11]
 [-2.36675458e-09 -2.89578563e-09  2.77846909e-11]
 [-2.54378391e-09 -3.10673977e-09  9.64729672e-11]
 [-3.59089941e-09 -4.36386010e-09  5.19960580e-11]
 [-4.88049854e-09 -6.08561558e-09 -9.18430207e-11]
 [-5.17103649e-09 -6.56385173e-09  3.13181446e-11]
 [-5.553308

-61.873399795796125
[[ 6.37641381e-09  6.60339730e-09 -1.08234308e-10]
 [ 5.83656236e-09  6.12553961e-09  2.39073067e-12]
 [ 6.30495000e-09  6.58194486e-09 -4.79582909e-11]
 [ 4.76035210e-09  5.22916332e-09 -3.68901957e-11]
 [ 4.36656347e-09  4.62868975e-09 -5.10026308e-11]
 [ 3.88809865e-09  4.14785460e-09 -4.19569325e-11]
 [ 2.28314535e-09  2.37309523e-09  6.94399145e-11]
 [ 1.90899834e-09  1.98447492e-09 -8.73430270e-12]
 [ 5.56549040e-10  5.95051728e-10  2.20051817e-11]
 [ 2.19046734e-10  2.17421435e-10 -9.24532804e-12]
 [-4.70906086e-11 -4.67853855e-11  2.60135851e-12]
 [-1.48886271e-10 -1.43495151e-10  2.31084035e-12]
 [-2.01835740e-10 -1.94278526e-10  1.41859122e-11]
 [-2.14037329e-10 -1.93920750e-10  1.17450337e-11]
 [-2.03514967e-10 -1.98076960e-10  3.97222706e-12]
 [-2.09398414e-10 -2.03219942e-10  4.36947811e-12]
 [-2.60592946e-10 -2.34720915e-10  1.05998980e-11]
 [-3.04937480e-10 -2.98639688e-10  1.80621267e-12]
 [-3.37405462e-10 -3.40380689e-10 -1.02483940e-11]
 [-3.936655

72.06598504605722
[[-1.09480157e-09 -2.11786720e-09  1.67042426e-10]
 [-3.94295828e-09 -8.25114045e-09 -1.33218590e-09]
 [ 5.07301376e-09 -0.00000000e+00  6.35024586e-09]
 [-0.00000000e+00 -0.00000000e+00  3.52334273e-10]
 [-2.00746088e-09 -8.23922117e-09  7.14014305e-10]
 [ 1.73955146e-10  1.17438990e-09  6.47034656e-11]
 [-3.93728416e-10 -1.85221878e-09  1.23236887e-09]
 [-0.00000000e+00 -0.00000000e+00 -1.61903186e-10]
 [ 5.94824671e-09  4.25559109e-09 -3.89924736e-11]
 [ 5.86061153e-09  3.18742165e-09 -7.49015733e-10]
 [ 3.01954891e-09  1.87431216e-09 -5.39948614e-10]
 [ 5.83583561e-10  5.73090560e-10 -1.09346336e-10]
 [ 2.83170780e-09  1.69496868e-09 -6.88929720e-11]
 [ 1.16386942e-09  2.99189477e-10 -1.93037029e-10]
 [ 1.54309187e-09  2.34471877e-10  7.26542532e-11]
 [ 1.67650916e-09  3.33637981e-10  8.85998739e-12]
 [ 1.43770001e-09  3.01811718e-10 -1.46632141e-10]
 [ 1.58917048e-09  9.50904256e-10 -8.30046064e-11]
 [ 2.14280224e-09  1.81518890e-09 -1.73398839e-10]
 [ 1.61560574

In [34]:
np.savetxt("optimizedresult-multipillar.txt", np.array(optresult))

In [34]:
NPbasinarray = np.array(basinarray)
NPretvalarray = np.array(retvalarray)
NPbest_fit_array = np.array(best_fit_array)
np.savetxt("basinarray_nonperiodic-17.txt", np.array(NPbasinarray))
np.savetxt("retvalarray_nonperiodic-17.txt", np.array(NPretvalarray))
np.savetxt("best_fit_array-17.txt", np.array(NPbest_fit_array))

In [35]:
np.savetxt("basinarray_nonperiodic-17.txt", np.array(NPbasinarray))
np.savetxt("retvalarray_nonperiodic-17.txt", np.array(NPretvalarray))
np.savetxt("best_fit_array-17.txt", np.array(NPbest_fit_array))
best_fit_array

[array([0.00871637]),
 array([0.07331751]),
 array([0.08495526]),
 array([0.10849658]),
 array([0.11787779]),
 array([0.14935133]),
 array([0.14024098]),
 array([0.13308344]),
 array([0.12228055]),
 array([0.10879259]),
 array([0.08823656]),
 array([0.67305569]),
 array([0.66660726]),
 array([0.58562107]),
 array([0.59441182])]

In [33]:
NPretvalarray

array([0.00745641, 0.26754927, 0.04489959, 0.02045703, 0.01086351,
       0.00745641, 0.01055321, 0.0071797 , 0.12315958, 0.31251066])

In [34]:
 np.array(np.loadtxt("retvalarray_multi-optimization.txt"))

FileNotFoundError: retvalarray_multi-optimization.txt not found.

In [35]:
max_Periods_array[-1]/max_Periods_array[0]

1.0

In [34]:
idx = np.argmin(NPretvalarray)
Ropt = NPbasinarray[idx]
idx

7

In [33]:
H = 15

In [35]:
e = np.array(np.loadtxt("final-multi-pillar-optimization.txt"))

In [35]:
e = e[:, H:Periods-H]

In [36]:
e.shape

(12, 30)

In [41]:
e = e[:2]

In [37]:
%matplotlib notebook

<IPython.core.display.Javascript object>


In [38]:
from matplotlib.animation import FuncAnimation
from matplotlib import animation
fig, ax = plt.subplots()

line, = ax.plot([])

XX = np.reshape(rguess1, (Periods, number_of_params))[:,2]
#XX = XX[H:Periods-H]

plt.plot(XX, d_n, 'o')
def animate_func(num):
    plt.plot(XX, e[num],)
    #plt.ylim((0.0, 0.30))
line_ani = animation.FuncAnimation(fig, animate_func, interval=500)
plt.show()

In [31]:
A = np.reshape(rguess1, (Periods, number_of_params))[:,2]

In [33]:
plt.plot(B[8:52], best_Gn[8:52])
plt.plot(B[8:52], d_n[8:52], 'o')

In [ ]:
#beta    e1
#0.4 -> 0.31

In [40]:
best_Gn

array([0.15949769, 0.15301201, 0.15242795, 0.1423946 , 0.13154756,
       0.12086706, 0.10255463, 0.0853472 , 0.07320302, 0.04436008,
       0.04220913, 0.01340642, 0.02335113, 0.00963061, 0.01422547,
       0.01274747, 0.01393683, 0.02034333, 0.02471299, 0.03696593,
       0.05226741, 0.06799516, 0.08466149, 0.10676502, 0.11669703,
       0.14321924, 0.14298518, 0.16314542, 0.15566976, 0.16286808])

In [39]:
np.savetxt("Final-multipillar-bestGvals-30per-1.txt", np.array(best_Gn))

In [35]:
H = 10

(150, 1)

In [36]:
XX = np.reshape(rguess1, (Periods, number_of_params))[:,2]
XX = np.linspace(0, Periods, Periods)
XX = XX[H:Periods-H]

plt.plot(XX, d_n[H:Periods-H], 'o')
#plt.plot(XX, best_Gn[H:Periods-H])